In [9]:


# 구글 코랩용 뉴스 크롤링 + 워드클라우드 예제

# 워드클라우드 관련 라이브러리 설치 (코랩에서 한 번만 실행)
!pip install wordcloud konlpy

# 1. 필수 라이브러리 설치
!pip install streamlit -q

# 나눔고딕 폰트 설치 및 설정
!apt-get update -qq
!apt-get install fonts-nanum -qq
!fc-cache -fv
!rm ~/.cache/matplotlib -rf


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/nanum: skipping, looped dire

In [10]:
# streamlit 화일명
%%writefile news_graph.py

# 필요한 라이브러리 임포트
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

# 폰트 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

Overwriting news_graph.py


# 코드시작

In [11]:
%%writefile news_graph.py

import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
import streamlit as st # Import streamlit

# 폰트 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False


def crawl_naver_news():
    """네이버 뉴스 메인 페이지에서 헤드라인을 크롤링하는 함수"""

    st.write(" 네이버 뉴스 크롤링을 시작합니다...") # Use st.write instead of print

    # 웹 페이지 URL
    url = "https://news.naver.com"

    # 헤더 설정 (코랩 환경에 맞게 조정)
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        # 웹 페이지 요청
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # HTML 파싱
        soup = BeautifulSoup(response.text, 'html.parser')

        # 뉴스 헤드라인 추출
        headlines = []

        # 여러 가지 방법으로 뉴스 헤드라인 찾기
        selectors = [
            'a.cjs_news_link',
            'a[href*="/article/"]',
            '.hdline_article_tit',
            '.cluster_text_headline'
        ]

        for selector in selectors:
            items = soup.select(selector)
            if items:
                for item in items[:10]:
                    title = item.get_text().strip()
                    if title and len(title) > 10:
                        headlines.append(title)
                break

        # 결과 출력
        st.write(f"\n 총 {len(headlines)}개의 뉴스 헤드라인을 찾았습니다!") # Use st.write
        st.write("=" * 60) # Use st.write

        for i, headline in enumerate(headlines[:10], 1):
            st.write(f"{i:2d}. {headline}") # Use st.write

        st.write("=" * 60) # Use st.write

        return headlines

    except requests.RequestException as e:
        st.error(f" 네트워크 오류: {e}") # Use st.error for errors
        return []
    except Exception as e:
        st.error(f" 크롤링 오류: {e}") # Use st.error
        return []


def create_news_dataframe(headlines):
    """뉴스 헤드라인을 데이터프레임으로 변환"""

    if not headlines:
        st.write(" 분석할 헤드라인이 없습니다.") # Use st.write
        return None

    # 데이터프레임 생성
    df = pd.DataFrame({
        '순번': range(1, len(headlines) + 1),
        '헤드라인': headlines,
        '글자수': [len(headline) for headline in headlines],
        '크롤링시간': [datetime.now().strftime('%Y-%m-%d %H:%M:%S')] * len(headlines)
    })

    st.write("\n 뉴스 데이터 분석:") # Use st.write
    st.write(f"총 뉴스 개수: {len(df)}") # Use st.write
    st.write(f"평균 글자수: {df['글자수'].mean():.1f}") # Use st.write
    st.write(f"최대 글자수: {df['글자수'].max()}") # Use st.write
    st.write(f"최소 글자수: {df['글자수'].min()}") # Use st.write

    return df

def visualize_news_data(df):
    """뉴스 데이터 시각화"""

    if df is None or df.empty:
        st.write(" 시각화할 데이터가 없습니다.") # Use st.write
        return

    # 그래프 스타일 설정
    plt.style.use('default')
    fig, axes = plt.subplots(1, 3, figsize=(20, 5))

    # 1. 헤드라인 글자수 분포 (히스토그램)
    axes[0].hist(df['글자수'], bins=10, color='skyblue', alpha=0.7, edgecolor='black')
    axes[0].set_title('News Headline Length Distribution', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Number of Characters')
    axes[0].set_ylabel('Frequency')
    axes[0].grid(True, alpha=0.3)

    # 2. 헤드라인별 글자수 (라인 그래프)
    axes[1].plot(df['순번'], df['글자수'], marker='o', color='lightcoral', alpha=0.7)
    axes[1].set_title('Character Count by Headline (Line)', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Headline Number')
    axes[1].set_ylabel('Number of Characters')
    axes[1].grid(True, alpha=0.3)

    # 3. 헤드라인별 글자수 (파이 그래프)
    total_chars = df['글자수'].sum()
    sizes = df['글자수'] / total_chars
    labels = [f'Headline {i}' for i in df['순번']]

    axes[2].pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    axes[2].set_title('Proportion of Total Characters by Headline', fontsize=14, fontweight='bold')
    axes[2].axis('equal')

    plt.tight_layout()
    st.pyplot(fig) # Display the matplotlib figure in Streamlit


# Streamlit 앱의 메인 로직
st.title("네이버 뉴스 헤드라인 분석") # Set a title for the Streamlit app

st.write(" 구글 코랩 웹 크롤링 실습을 시작합니다!")
st.write("=" * 70)

# 2. 실제 웹사이트 크롤링
headlines = crawl_naver_news()

# 4. 데이터 분석
if headlines:
    df = create_news_dataframe(headlines)

    if df is not None:
        st.write("\n 데이터프레임 미리보기:")
        st.dataframe(df.head()) # Use st.dataframe to display the dataframe

        # 5. 시각화
        visualize_news_data(df)

        # 6. CSV 파일로 저장 (코랩에서 다운로드 가능)
        df.to_csv('news_headlines.csv', index=False, encoding='utf-8')
        st.write("\n 'news_headlines.csv' 파일로 저장되었습니다!")
        st.write("   (코랩 왼쪽 파일 탭에서 다운로드 가능)")

st.write("\n 크롤링 실습이 완료되었습니다!")

Overwriting news_graph.py


In [13]:

from news_graph import crawl_naver_news, create_news_dataframe, visualize_news_data

# 메인 실행 함수
def main():
    """메인 실행 함수"""

    print(" 구글 코랩 웹 크롤링 실습을 시작합니다!")
    print("=" * 70)

    # 1. 기초 개념 데모
    #demo_web_scraping_basics()

    print("\n" + "="*70)

    # 2. 실제 웹사이트 크롤링
    headlines = crawl_naver_news()

    # 3. 간단한 테스트 사이트 크롤링
    #crawl_simple_news_site()

    # 4. 데이터 분석
    if headlines:
        df = create_news_dataframe(headlines)

        if df is not None:
            print("\n 데이터프레임 미리보기:")
            print(df.head())

            # 5. 시각화
            visualize_news_data(df)

            # 6. CSV 파일로 저장 (코랩에서 다운로드 가능)
            df.to_csv('news_headlines.csv', index=False, encoding='utf-8')
            print("\n 'news_headlines.csv' 파일로 저장되었습니다!")
            print("   (코랩 왼쪽 파일 탭에서 다운로드 가능)")

    print("\n 크롤링 실습이 완료되었습니다!")



# 코랩에서 바로 실행
if __name__ == "__main__":
    main()

2025-06-30 05:15:47.486 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-30 05:15:47.491 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-30 05:15:47.494 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


 구글 코랩 웹 크롤링 실습을 시작합니다!



2025-06-30 05:15:48.910 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-30 05:15:48.916 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-30 05:15:48.919 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-30 05:15:48.922 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-30 05:15:48.927 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-30 05:15:48.935 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-30 05:15:48.948 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-30 05:15:48.949 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar


 데이터프레임 미리보기:
   순번                                               헤드라인  글자수  \
0   1         문화유산 ‘성북동 별서’ 화재…소방, 대응 1단계 발령\n\n\n\n세계일보   38   
1   2  서울 성북구 소재 문화유산 '성북동 별서' 화재, 대응 1단계\n\n\n\nSBS ...   52   
2   3         문화유산 '성북동 별서' 화재…소방, 대응 1단계 발령\n\n\n\n한국경제   38   
3   4        한국 3대 정원 ‘성북동 별서’서 화재…소방당국 진화 중\n\n\n\n서울경제   39   
4   5           서울 성북구 소재 문화유산 ‘성북동 별서’에서 화재\n\n\n\n매일경제   36   

                 크롤링시간  
0  2025-06-30 05:15:49  
1  2025-06-30 05:15:49  
2  2025-06-30 05:15:49  
3  2025-06-30 05:15:49  
4  2025-06-30 05:15:49  


2025-06-30 05:15:50.342 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-30 05:15:51.889 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-30 05:15:51.890 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-30 05:15:51.891 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.



 'news_headlines.csv' 파일로 저장되었습니다!
   (코랩 왼쪽 파일 탭에서 다운로드 가능)

 크롤링 실습이 완료되었습니다!


In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [ ]:
!streamlit run news_graph.py & npx localtunnel --port 8501